In [ ]:
import json
import csv
import re
from typing import Dict, List, Tuple

class WordChecker:
    def __init__(self, json_file_path: str, csv_file_path: str):
        """
        Khởi tạo WordChecker
        
        Args:
            json_file_path: Đường dẫn đến file JSON
            csv_file_path: Đường dẫn đến file CSV
        """
        self.json_file_path = json_file_path
        self.csv_file_path = csv_file_path
        self.json_data = {}
        self.csv_data = {}
        
    def load_json_data(self) -> Dict:
        """Tải dữ liệu từ file JSON"""
        try:
            with open(self.json_file_path, 'r', encoding='utf-8') as f:
                self.json_data = json.load(f)
            print(f"✓ Đã tải thành công {len(self.json_data)} entries từ file JSON")
        except Exception as e:
            print(f"✗ Lỗi khi đọc file JSON: {e}")
        return self.json_data
    
    def load_csv_data(self) -> Dict:
        """Tải dữ liệu từ file CSV"""
        try:
            with open(self.csv_file_path, 'r', encoding='utf-8') as f:
                csv_reader = csv.DictReader(f)
                for row in csv_reader:
                    word_id = row['word_id']
                    self.csv_data[word_id] = {
                        'word': row['word'],
                        'synset_id': row['synset_id'],
                        'pos': row['pos'],
                        'gloss': row['gloss']
                    }
            print(f"✓ Đã tải thành công {len(self.csv_data)} từ từ file CSV")
        except Exception as e:
            print(f"✗ Lỗi khi đọc file CSV: {e}")
        return self.csv_data
    
    def check_word_in_sentences(self, word: str, sentences: List[str]) -> Tuple[bool, List[str]]:
        """
        Kiểm tra xem từ có xuất hiện trong danh sách câu không
        
        Args:
            word: Từ cần kiểm tra
            sentences: Danh sách các câu
            
        Returns:
            Tuple(có xuất hiện hay không, danh sách câu chứa từ đó)
        """
        found_sentences = []
        
        # Tạo pattern để tìm từ (word boundary để tránh tìm từ con)
        pattern = r'\b' + re.escape(word) + r'\b'
        
        for sentence in sentences:
            if re.search(pattern, sentence, re.IGNORECASE):
                found_sentences.append(sentence)
        
        return len(found_sentences) > 0, found_sentences
    
    def check_all_words(self) -> Dict:
        """
        Kiểm tra tất cả các từ trong CSV có xuất hiện trong JSON tương ứng không
        
        Returns:
            Dictionary chứa kết quả kiểm tra
        """
        results = {
            'found': [],
            'not_found': [],
            'missing_word_id': []
        }
        
        for word_id, word_info in self.csv_data.items():
            word = word_info['word']
            
            # Kiểm tra xem word_id có tồn tại trong JSON không
            if word_id not in self.json_data:
                results['missing_word_id'].append({
                    'word_id': word_id,
                    'word': word,
                    'reason': f'word_id {word_id} không tồn tại trong file JSON'
                })
                continue
            
            # Lấy danh sách câu từ JSON
            sentences = self.json_data[word_id]
            
            # Kiểm tra từ có xuất hiện trong câu không
            found, found_sentences = self.check_word_in_sentences(word, sentences)
            
            if found:
                results['found'].append({
                    'word_id': word_id,
                    'word': word,
                    'total_sentences': len(sentences),
                    'sentences_with_word': len(found_sentences),
                    'found_sentences': found_sentences
                })
            else:
                results['not_found'].append({
                    'word_id': word_id,
                    'word': word,
                    'sentences': sentences
                })
        
        return results
    
    def print_summary(self, results: Dict):
        """In tóm tắt kết quả"""
        print("\n" + "="*60)
        print("                    TÓM TẮT KẾT QUẢ")
        print("="*60)
        
        total_words = len(self.csv_data)
        found_count = len(results['found'])
        not_found_count = len(results['not_found'])
        missing_id_count = len(results['missing_word_id'])
        
        print(f"📊 Tổng số từ cần kiểm tra: {total_words}")
        print(f"✅ Số từ tìm thấy trong câu: {found_count}")
        print(f"❌ Số từ KHÔNG tìm thấy trong câu: {not_found_count}")
        print(f"⚠️  Số word_id không tồn tại trong JSON: {missing_id_count}")
        print(f"📈 Tỷ lệ tìm thấy: {found_count/max(total_words-missing_id_count, 1)*100:.1f}%")
    
    def print_detailed_results(self, results: Dict, show_found: bool = True, show_not_found: bool = True):
        """In kết quả chi tiết"""
        
        if show_found and results['found']:
            print("\n" + "="*60)
            print("✅ CÁC TỪ TÌM THẤY TRONG CÂU")
            print("="*60)
            for item in results['found']:
                print(f"\n🔍 Word ID: {item['word_id']} | Từ: '{item['word']}'")
                print(f"   📝 Tổng số câu: {item['total_sentences']}")
                print(f"   ✨ Số câu chứa từ: {item['sentences_with_word']}")
                print("   📄 Các câu chứa từ:")
                for i, sentence in enumerate(item['found_sentences'], 1):
                    print(f"      {i}. {sentence}")
        
        if show_not_found and results['not_found']:
            print("\n" + "="*60)
            print("❌ CÁC TỪ KHÔNG TÌM THẤY TRONG CÂU")
            print("="*60)
            for item in results['not_found']:
                print(f"\n🔍 Word ID: {item['word_id']} | Từ: '{item['word']}'")
                print("   📄 Các câu trong JSON:")
                for i, sentence in enumerate(item['sentences'], 1):
                    print(f"      {i}. {sentence}")
        
        if results['missing_word_id']:
            print("\n" + "="*60)
            print("⚠️  CÁC WORD_ID KHÔNG TỒN TẠI TRONG JSON")
            print("="*60)
            for item in results['missing_word_id']:
                print(f"   Word ID: {item['word_id']} | Từ: '{item['word']}'")
                print(f"   Lý do: {item['reason']}")
    
    def run_check(self, show_details: bool = True, show_found: bool = True, show_not_found: bool = True):
        """
        Chạy kiểm tra hoàn chỉnh
        
        Args:
            show_details: Hiển thị kết quả chi tiết
            show_found: Hiển thị các từ tìm thấy
            show_not_found: Hiển thị các từ không tìm thấy
        """
        print("🚀 Bắt đầu kiểm tra...")
        
        # Tải dữ liệu
        self.load_json_data()
        self.load_csv_data()
        
        if not self.json_data or not self.csv_data:
            print("❌ Không thể tải dữ liệu. Vui lòng kiểm tra đường dẫn file.")
            return None
        
        # Thực hiện kiểm tra
        print("\n🔍 Đang kiểm tra...")
        results = self.check_all_words()
        
        # In kết quả
        self.print_summary(results)
        
        if show_details:
            self.print_detailed_results(results, show_found, show_not_found)
        
        return results

# Hàm sử dụng
def main():
    """Hàm chính để chạy tool"""
    # Đường dẫn tới các file
    json_file = "pseudo_sent.json"  # Thay đổi đường dẫn này
    csv_file = "word_synsets_with_pos_with_gloss.csv"   # Thay đổi đường dẫn này
    
    # Tạo và chạy checker
    checker = WordChecker(json_file, csv_file)
    results = checker.run_check(
        show_details=True,      # Hiển thị chi tiết
        show_found=True,        # Hiển thị từ tìm thấy
        show_not_found=True     # Hiển thị từ không tìm thấy
    )
    
    return results

if __name__ == "__main__":
    # Chạy tool
    main()
    
    # Hoặc bạn có thể sử dụng như sau:
    # checker = WordChecker("path_to_json.json", "path_to_csv.csv")
    # results = checker.run_check()

🚀 Bắt đầu kiểm tra...
✗ Lỗi khi đọc file JSON: [Errno 2] No such file or directory: 'data.json'
✗ Lỗi khi đọc file CSV: [Errno 2] No such file or directory: 'words.csv'
❌ Không thể tải dữ liệu. Vui lòng kiểm tra đường dẫn file.


In [25]:
import json
import csv

# Đọc dữ liệu từ file JSON
json_file_path = "pseudo_sent.json"
with open(json_file_path, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# Đọc dữ liệu từ file CSV
csv_file_path = "word_synsets_with_pos_with_gloss.csv"
csv_rows = []
word_dict = {}

with open(csv_file_path, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    fieldnames = reader.fieldnames  # Lưu tên các cột
    
    for row in reader:
        csv_rows.append(row)
        word_dict[row['word_id']] = row['word']  # Lưu ánh xạ word_id -> word

# Tìm các word_id không có trong ít nhất một câu
problematic_ids = set()

for word_id, sentences in json_data.items():
    if word_id in word_dict:
        target_word = word_dict[word_id]
        
        for sentence in sentences:
            # Kiểm tra từ không tồn tại trong câu (không phân biệt hoa thường)
            if target_word.lower() not in sentence.lower():
                problematic_ids.add(word_id)
                break  # Chỉ cần 1 câu không có từ là đủ

# Lọc các dòng CSV có word_id bị lỗi
problematic_rows = [row for row in csv_rows if row['word_id'] in problematic_ids]

# Ghi kết quả ra file CSV (với tất cả các cột)
output_file = "missing_words_full.csv"
with open(output_file, 'w', encoding='utf-8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(problematic_rows)

print(f"Hoàn thành! Đã tìm thấy {len(problematic_ids)} từ có vấn đề.")
print(f"Kết quả đã được lưu vào {output_file} với đầy đủ các cột.")

Hoàn thành! Đã tìm thấy 15165 từ có vấn đề.
Kết quả đã được lưu vào missing_words_full.csv với đầy đủ các cột.
